# Parallel workflows with QiskitFunction

In this document, we will learn how to run distributed workflows inside a function. In this case, we will compute the quasi-probability distribution in parallel for a list of quantum circuits.

Let's take a look at the function file [./source_files/function_with_parallel_workflow.py](./source_files/function_with_parallel_workflow.py). 

```python

from qiskit_serverless import get_arguments, save_result, distribute_task, get

from qiskit import QuantumCircuit
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_ibm_runtime.fake_provider import FakeAlmadenV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# The @distribute_task decorator marks this function to be executed on a remote worker.
# It transpiles a circuit for a specific backend, runs it, and returns the results.
@distribute_task()
def distributed_compile_and_sample(ideal_circuit: QuantumCircuit, shots: int):
    """
    A distributed task that compiles a circuit for a specific backend ISA,
    runs it, and returns the measurement counts.
    """
    backend = FakeAlmadenV2()
    pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
    sampler = Sampler(mode=backend)
    isa_circuit=pm.run(ideal_circuit)
    pub_result = sampler.run([isa_circuit], shots=shots).result()[0]
    return pub_result.data.meas.get_counts()


if __name__ == "__main__":
    # Get input arguments, which include a list of circuits to process.
    arguments = get_arguments()
    circuits = arguments.get("circuits")
    shots = arguments.get("shots", 4096) 

    # Submit a remote task for each circuit. This returns immediately with references to the tasks.
    task_references = [
        distributed_compile_and_sample(circuit, shots) for circuit in circuits
    ]

    # Wait for all the distributed tasks to complete and retrieve their results.
    results = get(task_references)
    
    # Save the final aggregated results.
    save_result({"results": results})
```

There are a lot of new concepts introduced in this Qiskit Function, so let's go over them in more detail:

The [distribute_task](https://qiskit.github.io/qiskit-serverless/stubs/qiskit_serverless.core.distribute_task.html#qiskit_serverless.core.distribute_task) decorator converts a function into a distributed task. This means that the function will be executed on compute resources asynchronously and in parallel to the main context of the Qiskit Function.

When you call a converted function, it will return a reference to the function execution instead of the result. In order to get the result back, you need to call the [get](https://qiskit.github.io/qiskit-serverless/stubs/qiskit_serverless.core.get.html#qiskit_serverless.core.get) function on the function reference. `get` will wait until the function is finished and return the result of the function execution.

In the Qiskit Function above, we have applied the `distribute_task` decorator to the `distributed_sample` function. This function takes a `QuantumCircuit` as input and returns the quasi distribution for that circuit.

After we have defined the `distributed_sample` function, we read the circuits from the Qiskit Function arguments using the [get_arguments](https://qiskit.github.io/qiskit-serverless/stubs/qiskit_serverless.serializers.get_arguments.html#qiskit_serverless.serializers.get_arguments) function. We then call the `distributed_sample` function for each of the circuits, which creates a reference to each of the function executions.

These function executions will run in parallel on compute resources, and we get task references as the return type. We store these task references in the `sample_task_references` list.

After we have created the task references for each of the function executions, we need to collect the results from these tasks. We do this by calling the `get` function on the list of task references, which waits until all the tasks have completed and returns the results.

Once we have the results, we can save them using the [save_result](https://qiskit.github.io/qiskit-serverless/stubs/qiskit_serverless.core.save_result.html#qiskit_serverless.core.save_result) function.

Essentially, this Qiskit Function reads the circuits from the Qiskit Function arguments, executes the `distributed_sample` function on each circuit in parallel, collects the results from the function executions, and saves the results.

> &#x26A0; This provider is set up with default credentials to a test cluster intended to run on your machine. For information on setting up infrastructure on your local machine, check out the guide on [local infrastructure setup](https://qiskit.github.io/qiskit-serverless/deployment/local.html).

In [1]:
from qiskit_serverless import ServerlessClient
import os

client = ServerlessClient(
    token=os.environ.get("GATEWAY_TOKEN", "awesome_token"),
    host=os.environ.get("GATEWAY_HOST", "http://localhost:8000"),
    # If you are using the kubernetes approach the URL must be http://localhost
)

client

<gateway-client>

Let's create a list of random circuits which we will be passed as arguments to the function.

In [2]:
from qiskit.circuit.random import random_circuit

circuits = [random_circuit(2, 2) for _ in range(2)]
[circuit.measure_all() for circuit in circuits]
circuits

Run the function as usual, but pass the circuits in as a keyword argument, `circuits`.

In [3]:
from qiskit_serverless import QiskitFunction

function = QiskitFunction(
    title="function-with-parallel-workflow",
    entrypoint="function_with_parallel_workflow.py",
    working_dir="./source_files/",
)

client.upload(function)

QiskitFunction(function-with-parallel-workflow)

In [4]:
parallel_function = client.get("function-with-parallel-workflow")
job = parallel_function.run(circuits=circuits)
job

<Job | 84b572a4-8520-4397-b4d1-dc4acd776604>

In [5]:
job.status()

'QUEUED'

In [6]:
job.result()

{'results': [{'00': 3983, '10': 113}, {'00': 4096}]}